## Import tools

In [47]:
import numpy as np
import pandas as pd

## Get the data

In [130]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv("iris.csv", skiprows=1, header=None, names=col_names)
data.head(10)

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


## Node class

In [131]:
# Node class to store the tree structure
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature  # index of the feature to split on
        self.threshold = threshold  # value to split the feature
        self.left = left  # left subtree (where feature <= threshold)
        self.right = right  # right subtree (where feature > threshold)
        self.value = value  # prediction value if leaf node

    def is_leaf_node(self):
        return self.value is not None

## Decision tree


In [146]:
import numpy as np
from collections import Counter

class DecisionTreeClassifier:
    def __init__(self):
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        num_labels = len(np.unique(y))

        # Stopping criteria
        if num_labels == 1 or n_samples < 2:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y, n_features)

        # Recursively build subtrees
        left_idx, right_idx = self._split(X[:, best_feature], best_threshold)
        left_subtree = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_subtree = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        return Node(best_feature, best_threshold, left_subtree, right_subtree)

    def _best_split(self, X, y, n_features):
        best_gain = -1
        split_idx, split_threshold = None, None
        for feature_idx in range(n_features):
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_idx
                    split_threshold = threshold
        return split_idx, split_threshold

    def _information_gain(self, y, X_column, split_threshold):
        # Parent entropy
        parent_entropy = self._entropy(y)

        # Generate split
        left_idx, right_idx = self._split(X_column, split_threshold)

        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0

        # Calculate weighted average child entropy
        n = len(y)
        n_left, n_right = len(left_idx), len(right_idx)
        e_left, e_right = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        # Information gain
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_threshold):
        left_idx = np.where(X_column <= split_threshold)[0]
        right_idx = np.where(X_column > split_threshold)[0]
        return left_idx, right_idx

    def _entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.is_leaf_node():
            print(tree.value)

        else:
            print("X_"+str(tree.feature), "<=", tree.threshold)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

## Train-Test split

In [147]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
Y = Y[:,0]
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

## Fit the model

## Print the tree


## Test the model 